In [1]:
import tensorflow as tf 
import numpy as np 
import random 
import time 
import os 
from PIL import Image 

In [39]:
#验证码图片的存放路径
captcha_image_path = r'C:\Users\chenlonghua.JW\Documents\GitHub\data\captcha\images'
#验证码图片的宽度
captcha_image_width = 160 
#验证码图片的高度
captcha_image_height = 60 

char_set_len =10 
captcha_len =4 

#60%的验证码图片放入训练集中
train_image_percent = 0.6 
#训练集，用于训练的验证码图片的文件名
training_image_name = [] 
#验证集，用于模型验证的验证码图片的文件名
validation_image_name = [] 

#存放训练好的模型的路径
model_save_path = r'C:\Users\chenlonghua.JW\Documents\GitHub\data\captcha\models'

def get_image_file_name(imgPath=captcha_image_path):
    fileName = [] 
    total = 0 
    for filePath in os.listdir(imgPath):
        captcha_name = filePath.split('\\')[-1]
        fileName.append(captcha_name)
        total +=1 
    return fileName,total 

'''
将验证码转换为训练时用的标签向量，维数是40 
例如，验证码是'0296',则对应的标签是
[1 0 0 0 0 0 0 0 0 0
0 0 1 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 1
0 0 0 0 0 0 1 0 0 0]
'''
def name2label(name):
    label = np.zeros(captcha_len * char_set_len)
    for i,c in enumerate(name):
        idx = i *char_set_len + ord(c) - ord('0')
        label[idx] =1 
    return label 

#取得验证码图片的数据以及它的标签
def get_data_and_label(fileName,filePath=captcha_image_path):
    pathName = os.path.join(filePath,fileName)
    img = Image.open(pathName)
    #转为灰度图
    img = img.convert('L')
    image_array = np.array(img)
    image_data = image_array.flatten()/255 
    image_label = name2label(fileName[0:captcha_len])
    return image_data,image_label 

#生成一个训练batch 
def get_next_batch(batchSize=32,trainOrTest='train',step=0):
    batch_data = np.zeros([batchSize,captcha_image_width * captcha_image_height])
    batch_label = np.zeros([batchSize,captcha_len * char_set_len])
    fileNameList = training_image_name 
    if trainOrTest =='validate':
        fileNameList = validation_image_name 
    
    totalNumber = len(fileNameList)
    indexStart = step * batchSize 
    for i in range(batchSize):
        index = (i +indexStart) %totalNumber 
        name =fileNameList[index]
        img_data,img_label = get_data_and_label(name)
        batch_data[i,:] = img_data 
        batch_label[i,:] = img_label 
    return batch_data,batch_label 

        
#构建卷积神经网络并训练
def train_data_with_CNN():
    #初始化权值
    def weight_variable(shape,name='weight'):
        init = tf.truncated_normal(shape,stddev=0.1)
        var = tf.Variable(initial_value=init,name=name)
        return var 
    #初始化偏置项
    def bias_variable(shape,name='bias'):
        init = tf.constant(0.1,shape=shape)
        var = tf.Variable(init,name=name)
        return var 
    #卷积
    def conv2d(x,W,name='conv2d'):
        return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME',name=name)
    #池化
    def max_pool_2(x,name='maxpool'):
        return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME',name=name)
    
    #输入层
    #请注意X的name，在测试model时会用到它
    X = tf.placeholder(tf.float32,[None,captcha_image_width * captcha_image_height],name='input-data')
    Y = tf.placeholder(tf.float32,[None,captcha_len * char_set_len],name='label-input')
    x_input = tf.reshape(X,[-1,captcha_image_height,captcha_image_width,1],name='x-input')  #shape=(?, 60, 160, 1)

    #dropout,防止过拟合
    #请注意keep_prob的name，在测试model时会用到它
    keep_prob = tf.placeholder(tf.float32,name='keep-prob')
    
    #第一层卷积
    W_conv1 = weight_variable([5,5,1,32],'W_conv1')
    B_conv1 = bias_variable([32],'B_conv1')
    conv1 = tf.nn.relu(conv2d(x_input,W_conv1,'conv1') + B_conv1)
    conv1 = max_pool_2(conv1,'conv1-pool')
    conv1 = tf.nn.dropout(conv1,keep_prob)   #shape=(?, 30, 80, 32)
    print('conv1:',conv1)
    
    #第二层卷积
    W_conv2 = weight_variable([5,5,32,64],'W_conv2')
    B_conv2 = bias_variable([64],'B_conv2')
    conv2 = tf.nn.relu(conv2d(conv1,W_conv2,'conv2') + B_conv2)
    conv2 = max_pool_2(conv2,'conv2-pool')
    conv2 = tf.nn.dropout(conv2,keep_prob)   #shape=(?, 15, 40, 64)
    print('conv2:',conv2)
    #第三层卷积
    W_conv3 = weight_variable([5,5,64,64],'W_conv3')
    B_conv3 = bias_variable([64],'B_conv3')
    conv3 = tf.nn.relu(conv2d(conv2,W_conv3,'conv3') + B_conv3)
    conv3 = max_pool_2(conv3,'conv3-pool')
    conv3 = tf.nn.dropout(conv3,keep_prob) #shape=(?, 8, 20, 64) 
    print('conv3:',conv3)
    #全连接层
    #每次池化后，图片的宽度和高度均缩小为原来的一半，讲过上面的三次池化，高度和宽度均缩小8倍
    W_fc1 = weight_variable([20*8*64,1024],'W_fc1')
    B_fc1 = bias_variable([1024],'B_fc1')
    fc1 = tf.reshape(conv3,[-1,20*8*64])
    fc1 = tf.nn.relu(tf.add(tf.matmul(fc1,W_fc1),B_fc1))
    fc1 = tf.nn.dropout(fc1,keep_prob)    #shape=(?, 1024)
    print('fc1:',fc1)
    #输出层
    W_fc2 = weight_variable([1024,captcha_len * char_set_len],'W_fc2')
    B_fc2 = bias_variable([captcha_len * char_set_len],'B_Fc2')
    output = tf.add(tf.matmul(fc1,W_fc2),B_fc2,'output')     #shape=(?, 40)
    print('output:',output)
    
    #损失函数
    loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y,logits=output))
    #优化函数
    optimizer = tf.train.AdamOptimizer(0.001).minimize(loss)
    
    predict =tf.reshape(output,[-1,captcha_len,char_set_len],name='predict')
    labels = tf.reshape(Y,[-1,captcha_len,char_set_len],name='labels')
    
    #预测结果
    #请注意predict_max_idx 的name，在测试model时会用到它 
    predict_max_idx = tf.argmax(predict,axis=2,name='predict_max_idx')
    labels_max_idx = tf.argmax(labels,axis=2,name='labels_max_idx')
    predict_correct_vec = tf.equal(predict_max_idx,labels_max_idx)
    accuracy = tf.reduce_mean(tf.cast(predict_correct_vec,tf.float32))
    
    saver = tf.train.Saver() 
    init = tf.global_variables_initializer()
    with tf.Session() as sess:
        sess.run(init)
        steps = 0 
        for epoch in range(6000):
            train_data,train_label = get_next_batch(64,'train',steps)
            sess.run(optimizer,feed_dict={X:train_data,Y:train_label,keep_prob:0.75})
            if steps %100==0:
                test_data,test_label = get_next_batch(100,'validate',steps)
                acc = sess.run(accuracy,feed_dict={X:test_data,Y:test_label,keep_prob:1.0})
                print('steps=%d,accuracy=%f' % (steps,acc))
                if acc >0.99:
                    saver.save(sess,model_save_path+'\\'+"crack_captcha.model", global_step=steps)
                    break 
            steps +=1 
    

In [40]:
if __name__=='__main__':
    image_filename_list ,total = get_image_file_name(captcha_image_path)
    random.seed(time.time())
    #打乱顺序
    random.shuffle(image_filename_list)
    trainImageNumner = int(total * train_image_percent)
    #测试集
    training_image_name = image_filename_list[:trainImageNumner]
    #验证集
    validation_image_name = image_filename_list[trainImageNumner:]
    
    train_data_with_CNN()
    print('training finished')
    

conv1: Tensor("dropout_50/mul:0", shape=(?, 30, 80, 32), dtype=float32)
conv2: Tensor("dropout_51/mul:0", shape=(?, 15, 40, 64), dtype=float32)
conv3: Tensor("dropout_52/mul:0", shape=(?, 8, 20, 64), dtype=float32)
fc1: Tensor("dropout_53/mul:0", shape=(?, 1024), dtype=float32)
output: Tensor("output_10:0", shape=(?, 40), dtype=float32)


KeyboardInterrupt: 